# Daily Queries 1

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect
from IPython.display import display

<a class="anchor" id="0_toc"></a>
# Table of Contents
***

1. [SQL Engine](#1-engine)
2. [Tables](#2-tables)
3. [Queries](#3-queries)
    1. [How many users got paid?](#q1)
    2. [Show the 5 users with the highest payouts](#q2)
    3. [Show the 5 users with the lowest payouts](#q3)
    4. [How much ether was paid out in November 2020?](#q4)
    5. [Which plan is the most popular?](#q5)
    6. [Which plan is the most popular amongst Linux users?](#q6)
    7. [What is the percentage of payout between the different plans?](#q7)
    8. [Users of which operating system earned more in payouts?](#q8)
    9. [What is the average payout amount per user for each of the OS in July 2020?](#q9)
    10. [What is the daily share of ether earned by users from Linux that are in the free plan in this data?](#q10)

<a class="anchor" id="1-engine"></a>
## SQL Engine
***
[back to Table of Contents](#0_toc)

In [2]:
db_name = 'data-analyst-eth-payouts-db'

In [3]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': db_name}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
inspector = inspect(engine)

In [4]:
def read_schema(table_name):
    return pd.DataFrame(inspector.get_columns(table_name)).rename_axis(table_name, axis=1)

In [5]:
def execute_query(q):
    return pd.io.sql.read_sql(q, con=engine)

<a class="anchor" id="2-tables"></a>
## Inspect tables
***
[back to Table of Contents](#0_toc)

In [6]:
tables = inspector.get_table_names()
tables

['payout', 'plan']

In [7]:
for table in tables:
    display(read_schema(table))

payout,name,type,nullable,default,autoincrement,comment
0,user_id,INTEGER,True,None,False,None
1,eth_address,TEXT,True,None,False,None
2,date,TIMESTAMP,True,None,False,None
3,payout,"NUMERIC(6, 5)",True,None,False,None


plan,name,type,nullable,default,autoincrement,comment
0,user_id,INTEGER,True,None,False,None
1,OS,TEXT,True,None,False,None
2,Plan,TEXT,True,None,False,None


In [8]:
for table in tables:
    display(execute_query(f'SELECT * FROM {table} LIMIT 1'))

,user_id,eth_address,date,payout
0,1005,0x4f9117b14426ac44ead4eaef0223830cb16bdb07,2020-07-27,1.68113


,user_id,OS,Plan
0,1005,Windows,Free


<a class="anchor" id="3-queries"></a>
## Queries
***
[back to Table of Contents](#0_toc)

1. [How many users got paid?](#q1)
2. [Show the 5 users with the highest payouts](#q2)
3. [Show the 5 users with the lowest payouts](#q3)
4. [How much ether was paid out in November 2020?](#q4)
5. [Which plan is the most popular?](#q5)
6. [Which plan is the most popular amongst Linux users?](#q6)
7. [What is the percentage of payout between the different plans?](#q7)
8. [Users of which operating system earned more in payouts?](#q8)
9. [What is the average payout amount per user for each of the OS in July 2020?](#q9)
10. [What is the daily share of ether earned by users from Linux that are in the free plan in this data?](#q10)

<a class="anchor" id="q1"></a>
### 1. How many users got paid?
[up](#3-queries)

In [9]:
execute_query("""
SELECT 
    COUNT(DISTINCT user_id)
FROM
    payout
WHERE
    payout > 0
""")

,count
0,433


<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q2"></a>
### 2. Show the 5 users with the highest payouts
[up](#3-queries)

In [10]:
execute_query("""
SELECT 
    user_id,
    SUM(payout."payout") as payout
FROM
    payout
GROUP BY
    user_id 
ORDER BY
    payout DESC
LIMIT 5
""")

,user_id,payout
0,1537,10.74169
1,3051,9.73121
2,1512,8.80816
3,1127,8.78861
4,4848,8.42445


<div class="alert alert-warning" role="alert">
Great but can you show per user instead of single transaction?
</div>

<div class="alert alert-info"> <b>Student comment v. 1:</b> <br />
    Updated
</div>

<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q3"></a>
### 3. Show the 5 users with the lowest payouts
[up](#3-queries)

In [11]:
execute_query("""
SELECT 
    user_id,
    SUM(payout."payout") as payout
FROM
    payout
GROUP BY
    user_id 
ORDER BY
    payout
LIMIT 5
""")[::-1]

,user_id,payout
4,4467,0.05056
3,2462,0.03381
2,2813,0.02914
1,3410,0.00818
0,2003,0.00775


<div class="alert alert-warning" role="alert">
Great but can you show per user instead of single transaction?
</div>

<div class="alert alert-info"> <b>Student comment v. 1:</b> <br />
    Updated
</div>

<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q4"></a>
### 4. How much ether was paid out in November 2020?
[up](#3-queries)

In [12]:
execute_query("""
SELECT 
    SUM(payout)
FROM
    payout
WHERE
    EXTRACT(MONTH FROM date) = 11
""")

,sum
0,166.0118


<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q5"></a>
### 5. Which plan is the most popular?
[up](#3-queries)

In [13]:
execute_query("""
SELECT
    plan."Plan" AS plan,
    COUNT(1) AS count
FROM
    plan
GROUP BY
    plan."Plan"
ORDER BY
    count DESC
--LIMIT 1
""")

,plan,count
0,Free,220
1,Premium,213


<div class="alert alert-success" role="alert">
Great! can you also show the other plan?</div>

<div class="alert alert-info"> <b>Student comment v. 1:</b> <br />
    I agree, I thought twice about removing the other values since it obscures the results, <br />
    but decided to do it in order to explicitly answer the question without extra text. <br /><br />
    In second thought, it would have been better to supplement the data with an additional explanatory markdown cell.
</div>

<div class="alert alert-success" role="alert">
V2: Exactly that way we can also check ourselves but it's ok to not show it if you just right a line about it and the reviwer can unserstand from the code</div>

<a class="anchor" id="q6"></a>
### 6. Which plan is the most popular amongst Linux users?
[up](#3-queries)

In [14]:
execute_query("""
SELECT
    plan."Plan" AS plan,
    COUNT(1) AS count
FROM
    plan
WHERE
    plan."OS" = 'Linux'
GROUP BY
    plan."Plan"
ORDER BY
    count DESC
--LIMIT 1
""")

,plan,count
0,Premium,76
1,Free,68


<div class="alert alert-success" role="alert">
Great! can you also show the other plan?</div>

<div class="alert alert-info"> <b>Student comment v. 1:</b> <br />
    Updated
</div>

<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q7"></a>
### 7. What is the percentage of payout between the different plans?
[up](#3-queries)

In [15]:
execute_query("""
SELECT
    *,
    ROUND(sum * 100 / SUM(sum) OVER (), 2) AS percent
FROM
    (SELECT
        plan."Plan" AS plan,
        SUM(payout."payout")
    FROM
        payout
            INNER JOIN plan ON plan.user_id = payout.user_id
    GROUP BY
        plan."Plan") AS subq
""")

,plan,sum,percent
0,Free,488.68999,53.97
1,Premium,416.76178,46.03


<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q8"></a>
### 8. Users of which operating system earned more in payouts?
[up](#3-queries)

In [16]:
execute_query("""
SELECT
    plan."OS" AS os,
    SUM(payout."payout") AS payout
FROM
    payout
        INNER JOIN plan ON plan.user_id = payout.user_id
GROUP BY
    plan."OS"
ORDER BY
    payout DESC
--LIMIT 1
""")

,os,payout
0,Linux,322.00353
1,Windows,316.03701
2,MAC,267.41123


<div class="alert alert-success" role="alert">
Great! but can you also show the number that was generated?</div>

<div class="alert alert-info"> <b>Student comment v. 1:</b> <br />
    It was like this before I've submitted 😄 <br />
    same explanation as in the previous comment.
</div>

<div class="alert alert-success" role="alert">
Great! - and same as before as well😉</div>

<a class="anchor" id="q9"></a>
### 9. What is the average payout amount per user for each of the OS in July 2020?
[up](#3-queries)

In [17]:
execute_query("""
SELECT
    plan."OS",
    AVG(payout."payout") AS avg
FROM
    payout
        INNER JOIN plan ON plan.user_id = payout.user_id
WHERE
    payout.date >= '2020-07-01' AND payout.date < '2020-08-01'
GROUP BY
    plan."OS"
ORDER BY
    avg DESC
""")

,OS,avg
0,Linux,1.931114
1,MAC,1.640537
2,Windows,1.590600


<div class="alert alert-success" role="alert">
Great!</div>

<a class="anchor" id="q10"></a>
### 10. What is the daily share of ether earned by users from Linux that are in the free plan in this data?
[up](#3-queries)

In [18]:
execute_query("""
SELECT
    subq.*,
    ROUND(subq."payout" / SUM(subq."payout") OVER (PARTITION BY subq.date), 2) AS "daily ratio"
FROM
    (SELECT
        payout.user_id,
        payout.date,
        SUM(payout.payout) AS payout
    FROM
        payout
            INNER JOIN plan ON plan.user_id = payout.user_id
    WHERE
        plan."OS" = 'Linux' AND plan."Plan" = 'Free'
    GROUP BY
        payout.date,
        payout.user_id) as subq
ORDER BY
    subq.date DESC,
    subq.payout DESC,
    subq.user_id
""")

,user_id,date,payout,daily ratio
0,4131,2021-01-05,2.36296,0.64
1,4608,2021-01-05,1.35791,0.36
2,2128,2021-01-04,0.68610,1.00
3,1330,2021-01-03,1.51269,1.00
4,2935,2021-01-02,0.30163,1.00
...,...,...,...,...
94,3542,2020-07-14,0.67543,1.00
95,4235,2020-07-13,3.13763,1.00
96,2154,2020-07-11,1.72359,1.00
97,3146,2020-07-08,1.39229,1.00


<div class="alert alert-success" role="alert">
Great!</div>